In [4]:
import pandas as pd
import numpy as np
import os
import ast
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import pickle
from flask import Flask, render_template, request, redirect, url_for, flash, jsonify, send_file
import io

In [ ]:
# Inicializar la aplicación Flask
app = Flask(__name__)
app.secret_key = "secret_key"  # Necesario para usar flash messages

#Selecciono modelo
nombre_modelo = 'RandomForest'

subcarpeta='models/serum'
tamaño_serum = 23
archivo = f'{nombre_modelo}_{tamaño_serum}.pkl'

# Cargar el modelo preentrenado (puedes usar pickle para cargarlo si lo guardaste así)
with open(f'./{subcarpeta}/{archivo}', 'rb') as file:
    model_serum = pickle.load(file)

#Selecciono modelo
subcarpeta='models/plasma'
tamaño_plasma = 24
archivo = f'{nombre_modelo}_{tamaño_plasma}.pkl'

# Cargar el modelo preentrenado (puedes usar pickle para cargarlo si lo guardaste así)
with open(f'./{subcarpeta}/{archivo}', 'rb') as file:
    model_plasma = pickle.load(file)

model = None
   
# Definir las columnas esperadas
COLUMNAS_ESPERADAS_S = ['Sex', 'Fluid', 'Exosome', 'hcmv-miR-US33-5p', 'hsa-miR-3165', 'hsa-miR-126-5p', 'hsa-miR-6881-3p', 'hsa-miR-6715b-3p',
                        'hsa-miR-4701-5p', 'hsa-miR-4715-5p', 'hsa-miR-7151-5p', 'hsa-miR-340-5p', 'hsa-miR-1193', 'hsa-miR-641',
                        'hsa-miR-449c-5p', 'hsa-miR-23b-3p', 'hsa-miR-323b-3p', 'hsa-miR-153-3p', 'hsa-miR-1293', 'hsa-miR-10a-3p',
                        'hsa-miR-27b-3p', 'hsa-miR-337-3p', 'hsa-miR-3131', 'hsa-miR-484', 'hsa-miR-369-5p', 'hsa-miR-7109-3p']

COLUMNAS_ESPERADAS_P = ['Sex', 'Fluid', 'Exosome', 'hsa-miR-6892-5p', 'hsa-miR-4753-5p', 'hsa-miR-124-3p', 'hsa-miR-2116-3p',
                        'hsa-miR-3940-3p', 'hsa-miR-384', 'hsa-miR-4433b-3p', 'hsa-miR-5100', 'hsa-miR-23b-3p', 'hsa-miR-9985',
                        'hsa-miR-5193', 'hsa-let-7f-5p', 'hsa-miR-6866-5p', 'hsa-miR-4508', 'hsa-miR-3150a-5p', 'hsa-miR-500a-3p',
                        'hsa-miR-4443', 'hsa-miR-1306-5p', 'hsa-miR-425-3p', 'hsa-miR-758-5p', 'hsa-miR-363-5p', 'hsa-miR-4799-5p',
                        'hsa-miR-144-5p', 'hsa-miR-216a-3p']

# Diccionario que asocia tipos de fluido con los tipos de cáncer que se pueden predecir
tipos_de_cancer_por_fluido = {
    'plasma': ['Healthy Control', 'Colorectal Cancer', 'Prostate Cancer',
               'Pancreatic Cancer', 'Metastatic renal cell carcinoma (mRCC)',
               'chronic fatigue syndrome', 'non-fatigued'],
    'serum': ['Serous', 'Mucinous', 'Endometrioid', 'Control', 'Endometrioma',
              'Endo-Clear Cell']
}

# Función para validar los datos
def cargar_y_validar_datos(datos):
    # 1. Verificar valores nulos
    if datos.isnull().values.any():
        return "Existen valores nulos en el dataset."
    
    # 2. Validar valores no válidos en columnas específicas
    valores_sexo_validos = ['male', 'female']
    valores_exosomas_validos = [True, False]
    valores_fluid_validos = ['plasma','serum']
    
    if not datos['Sex'].isin(valores_sexo_validos).all():
        return "La columna 'Sexo' contiene valores no válidos."
    if not datos['Exosome'].isin(valores_exosomas_validos).all():
        return "La columna 'Exosomas' contiene valores no válidos."
    if not datos['Fluid'].isin(valores_fluid_validos).all():
        return "La columna 'Fluid' contiene valores no válidos."

    # 3. Verificar si están todas las columnas clave
    if datos['Fluid'].unique()[0] == 'plasma':
        columnas_faltantes = [col for col in COLUMNAS_ESPERADAS_P if col not in datos.columns]
    else:
        columnas_faltantes = [col for col in COLUMNAS_ESPERADAS_S if col not in datos.columns]
    
    if columnas_faltantes:
        return f"Faltan las siguientes columnas:\n {', '.join(columnas_faltantes)}"
    
    return None

# Función para realizar la predicción
def predecir_tipo_cancer(datos):
    # Transformar las variables categóricas
    le_sexo = LabelEncoder()
    le_exosomas = LabelEncoder()
    
    datos['Sex_l'] = le_sexo.fit_transform(datos['Sex'])
    datos['Exosome_l'] = le_exosomas.fit_transform(datos['Exosome'])

    
    global model
    if datos['Fluid'].unique()[0] == 'plasma':
        features = datos.loc[:,COLUMNAS_ESPERADAS_P + ['Sex_l','Exosome_l']].copy()

        folder = './data/regularizacion'
        # Leer el archivo de texto que contiene los estadísticos
        with open(f'{folder}/regularizacion_plasma.txt', 'r', encoding='utf-8') as fn:
            contenido = fn.read()

        # Cambio a diccionario
        estadisticos = ast.literal_eval(contenido)
        # Establezco valor máximo y mínimo
        for col in estadisticos:
            try:
                # Limitar valores al rango
                features[col] = features[col].clip(lower=estadisticos[col][0], upper=estadisticos[col][1])
            except:
                pass
            
        # Inicializar el escalador MinMax
        scaler = MinMaxScaler()
        
        # Normalizar los datos del DataFrame
        variables_miR_norm = pd.DataFrame(scaler.fit_transform(features.iloc[:,3:]), columns=features.iloc[:,3:].columns)
        features = pd.concat([features.iloc[:,:3],variables_miR_norm],axis=1)
        #print (features.tail(1))
    
        features = features.iloc[:,list(range(3, tamaño_plasma+(3)))+[-2,-1]]
        #print(features.columns)
        model = model_plasma
        
    else:
        features = datos.loc[:,COLUMNAS_ESPERADAS_S].copy()

        folder = './data/regularizacion'
        # Leer el archivo de texto que contiene los estadísticos
        with open(f'{folder}/regularizacion_serum.txt', 'r', encoding='utf-8') as fn:
            contenido = fn.read()

        # Cambio a diccionario
        estadisticos = ast.literal_eval(contenido)
        # Establezco valor máximo y mínimo
        for col in estadisticos:
            try:
                # Limitar valores al rango
                features[col] = features[col].clip(lower=estadisticos[col][0], upper=estadisticos[col][1])
            except:
                pass
            
        # Inicializar el escalador MinMax
        scaler = MinMaxScaler()
        
        # Normalizar los datos del DataFrame
        variables_miR_norm = pd.DataFrame(scaler.fit_transform(features.iloc[:,3:]), columns=features.iloc[:,3:].columns)
        features = pd.concat([features.iloc[:,:3],variables_miR_norm],axis=1)
        #print (features.tail(1))
    
        features = features.iloc[:,range(3, tamaño_serum+3)]
        model = model_serum
    
    # Obtener probabilidades y predicción
    probabilidades = model.predict_proba(features)
    predicciones = model.predict(features)
    
    return probabilidades, predicciones

# ----------------------
# Ruta principal para la carga de archivos
@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        # Verificar si se ha subido un archivo
        if 'file' not in request.files:
            flash("No se ha subido ningún archivo", "error")
            return redirect(request.url)
        
        file = request.files['file']
        if file.filename == '':
            flash("El nombre del archivo está vacío", "error")
            return redirect(request.url)

        # Cargar el archivo CSV
        try:
            datos = pd.read_csv(file)
        except Exception as e:
            flash(f"Error al cargar el archivo: {str(e)}", "error")
            return redirect(request.url)
        
        # Verificar que la columna "Fluid" existe
        if 'Fluid' not in datos.columns:
            flash("El archivo no contiene una columna llamada 'Fluid'", "error")
            return redirect(request.url)
        
        # Validar los datos
        mensaje_error = cargar_y_validar_datos(datos)
        if mensaje_error:
            flash(mensaje_error, "error")
            return redirect(request.url)
        
        # Si los datos están correctos, realizar la predicción
        probabilidades, predicciones = predecir_tipo_cancer(datos)

        resultados = pd.DataFrame(np.zeros(len(datos)), columns=['Sample'])
        
        if 'Sample' in datos.columns.tolist():
            resultados['Sample'] = datos['Sample'].values
        else:
            resultados['Sample'] = range(len(datos))

        # Convertir a DataFrame para facilitar la visualización
        resultados[model.classes_] = probabilidades
        resultados['Predicción'] = predicciones
        resultados['Actual'] = datos['Desc'].values


        # Crear un archivo CSV temporal con los resultados
        resultados_csv = io.StringIO()
        resultados.to_csv(resultados_csv, index=False)
        resultados_csv.seek(0)

        # Guardar el CSV en un archivo temporal para descargar
        with open("resultados_prediccion.csv", "w") as f:
            f.write(resultados_csv.getvalue())
        
        # Obtener el fluido y los tipos de cáncer que se pueden predecir
        fluidos = datos['Fluid'].unique()
        mensaje_tipos_cancer = ""
        if fluidos[0] in tipos_de_cancer_por_fluido:
            tipos_cancer = tipos_de_cancer_por_fluido[fluidos[0]]
            mensaje_tipos_cancer = f"Con el fluido {fluidos[0]} se pueden predecir los siguientes tipos de cáncer: {', '.join(tipos_cancer)}."
        else:
            mensaje_tipos_cancer = f"El fluido '{fluidos[0]}' no es reconocido para predicciones."

        # Aplicar estilos para resaltar la columna mayor en azul
        def resaltar_maximo(s):
            if len(s) == len(datos.Desc.unique())+ 2:
                # Seleccionar el rango de columnas desde la segunda hasta la penúltima
                subset = s.iloc[1:-1]
                is_max = subset == subset.max()
                #print (is_max)
                # Retornar el estilo sólo para el rango de columnas, el resto sin estilo
                return ['' if i < 1 or i >= len(s)-1 else 'background-color: lightblue' if is_max[i-1] else '' for i in range(len(s))]
            else:
                # Seleccionar el rango de columnas desde la segunda hasta la penúltima
                subset = s.iloc[1:-2]
                is_max = subset == subset.max()
                #print (is_max)
                # Retornar el estilo sólo para el rango de columnas, el resto sin estilo
                return ['' if i < 1 or i >= len(s)-2 else 'background-color: lightblue' if is_max[i-1] else '' for i in range(len(s))]


        # Aplicar el estilo al DataFrame
        styled_resultados = resultados.style.apply(resaltar_maximo, axis=1)

        # Convertir el DataFrame a HTML sin el índice
        resultados_html = styled_resultados.hide(axis='index').set_table_attributes('class="table table-bordered"').to_html()
        #resultados_html = resultados.to_html(classes='table table-striped', index=False)
        return render_template("resultados.html", resultados=resultados_html, mensaje=mensaje_tipos_cancer)
    
    return render_template("index.html")


# Ruta para mostrar los resultados
@app.route("/resultados")
def resultados():
    return render_template("resultados.html")

@app.route("/descargar")
def descargar():
    # Enviar el archivo CSV al usuario para que lo descargue
    try:
        return send_file("resultados_prediccion.csv", as_attachment=True, download_name="resultados_prediccion.csv")
    except Exception as e:
        return str(e)

# Iniciar la aplicación
if __name__ == "__main__":
    # Solo para entornos interactivos como Jupyter o Spyder
    app.run(debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/Oct/2024 20:29:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2024 20:29:49] "GET /static/styles.css HTTP/1.1" 304 -
C:\Users\eagle\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\eagle\anaconda3\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
127.0.0.1 - - [17/Oct/2024 20:30:05] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [17/Oct/2024 20:30:05] "GET /static/styles.css HTTP/1.1" 304 -
